8 PUZZLE Using Simulated annealing

Importing all necessary libraries.

In [188]:
import numpy as np
import random
import time
import math
import copy
import sys
sys.stdout = open('log_a.txt', 'w')

In [189]:
def init():
    start = np.array([1, 2, 3, 4, 0, 6, 7, 5, 8])
    goal = np.array([1, 2, 3, 4, 5, 6, 7, 8, 0])
    np.random.shuffle(start)                                    # shuffling initial State
    start = np.reshape(start, (3, 3))
    goal = np.reshape(goal, (3, 3))
    return start, goal

Calculating energy using Misplaced Tiles.

In [190]:
def energy(s, g):
    count = 0
    for i in range(1,9):
        if (np.where(s == i) != np.where(g == i)):
            count += 1   
    return count

In [191]:
def moves(state):
    current_state = state.copy()
    y = {}
    z_r, z_c = np.where(current_state == 0)                          #index of '0'
    if z_r - 1 >= 0: 
        u_r, u_c = z_r - 1, z_c
        y["u"] = [u_r, u_c]                                          #UP
    if z_r + 1 <= 2:
        d_r, d_c = z_r + 1, z_c
        y["d"] = [d_r, d_c]                                          #DOWN
    if z_c - 1 >= 0:
        l_r, l_c = z_r, z_c - 1
        y["l"] = [l_r, l_c]                                          #LEFT
    if z_c + 1 <= 2:
        r_r, r_c = z_r, z_c + 1
        y["r"] = [r_r, r_c]                                          #RIGHT
    return y

In [192]:
def random_adj(state, goal):
    current_state = state.copy()                                    #Keeping a copy of start state
    direction = moves(state)                                        # getting all the possible moves
    cache = list(direction.keys())
    index = random.randrange(0, len(cache))
    r, c = direction[cache[index]]
    r, c = int(r), int(c) 
      
    z_r, z_c = np.where(current_state == 0)                         # Locating position of zero
    z_r, z_c = int(z_r), int(z_c)
    temp = current_state[r][c]
    current_state[z_r][z_c] = temp
    current_state[r][c] = 0
    dist = energy(current_state, goal)                              # Calcualting Energy of newly generated states
    return current_state, dist

In this Function we have mainly checked Delta Energy, i.e, Change in Energy.
Change in Energy = New Energy - Final Energy, if it's less than 0 ,  then it is acceptable.
Else, Generate probability as well as a random number.
Compare the probablity with random number , if it's less then do not reject it. Although its not feasible but we have to accept it as it might be useful in futures.

In [193]:
def sim_anl_energy_check(T, T_min, r, L):

    state, goal = init()
    print("The Goal state is:\n", goal)
    while T > T_min:
        
        for j in range(L):
            
            current_energy = energy(state, goal)
            print("The current state is:\n", state)
            print("The Energy in the current state is:", current_energy)
            if current_energy == 0: 
                return True
            new_state, new_energy = random_adj(state, goal) 
            print("The randomly generated new solution state is:\n", new_state)
            print("The Energy for randomly generating a new solution is:", new_energy)
            delta_energy = current_energy - new_energy
            if delta_energy> 0:
                print("This is a better solution, just accept it")
                state = new_state
            else:
                if math.exp(delta_energy / T)> random.random(): 
                    print("The probability of receiving a worse solution is:", math.exp(delta_energy / T))
                    print("This is a worse solution, but it was accepted")
                    state = new_state
            T = T - r
    return True 

In [195]:
def SA_algo(T, T_min, r, L, num):
    begin = time.time()
    Success = 0
    Failed = 0
    for i in range(num):
        if sim_anl_energy_check(T, T_min, r, L):
            Success += 1
            print("The {0}th example found the optimal solution".format(i))
        else:
            Failed += 1
            print("The {0}th example failed".format(i))
    end = time.time()
    print("The successfully resolved example of the {0} examples is: {1}".format(num, Success))
    print("The percentage of successful solutions for {0} examples is: {1}".format(num, Success / num))
    print("The failed example in {0} examples is: {1}".format(num, Failed))
    print("The percentage of failed cases in {0} is: {1}".format(num, Failed / num))
    print("The time required for {0} examples to run the algorithm is: {1} seconds".format(num, end-begin))
 
 
SA_algo(T=100, T_min=0.1, r=2, L=10, num=10)